## **Autores**

Saby Espinel - Cod: 201215868

Diego Salazar - Cod: 201628925

## Entrenando un clasificador
Se empleo un clasificador de noticias para noticias de Reuters.

In [2]:
import numpy as np
import keras
from keras.datasets import reuters
from keras.layers import Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer

max_words = 1000

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

Using TensorFlow backend.


Loading data...
2113536/2110848 [==============================] - 1s 0us/step
8982 train sequences
2246 test sequences
46 classes


In [3]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
num_words = max(word_index.values()) + 1
words = ['']*num_words
for word in word_index:
    words[word_index[word]] = word
print([words[i-2] for i in x_train[101][1:]])

557056/550378 [==============================] - 0s 1us/step
['', 'dlr', 'and', 'cts', '', '80', 'average', 'companies', 'in', 'income', 'of', 'make', '', '', 'said', '', '', 'a', 'of', 'make', '52', '', 'said', '', 'of', '1987', '', '2', 'of', 'sold', 'general', 'states', 'to', '', 'field', 'securities', 'was', 'agricultural', '', '3', 'it', 'a', '1988', 'said', 'as', 'april', '50', 'term', 'to', 'earlier', '3', 'it', 'but', 'was', 'with', '', 'said', '', 'previously', 'be', 'sell', 'cts', 'previously', 'be', '', 'more', 'earlier', 'of', 'which', 'and', 'said', 'commerce', 'of', '1987', 'was', '', 'august', '3', 'it', 'export', 'april', 'report', 'vice', 'to', 'beef', '3', 'it', '', 'and', '000', 'for']


Construccion de los sets de entrenamiento y validacion.

In [4]:
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (8982, 46)
y_test shape: (2246, 46)


________________________________________________________________________________________________________________________

## Modelo neuronal de capas densas
Se genero un modelo que tuviera los siguientes parametros: <br>
> -La primera capa oculta tiene **256** neuronas. <br>
> -Como parametros por defecto: activation=''**relu**'', optimizer="**SGD**", drop=**1**, kernel='**random_uniform**', neurons=**10** <br>
> -Si la entrada **moreHiddenLayers==False**, entonces se adicionaran  tres capas mas. Estas capas tambien tendran los paramtros de inicializacion establecidos anteriormente.

Se empleo la funcion de **KerasClassifier** el cual funciona como un wrapper entre "*Sklearn*" y "*keras*". Debido a esto se logro hacer una busqueda por grilla aleatoria (*randomSearchCV*) y fija (*gridSearchCV*). 

In [6]:
from keras.layers import Dense, Activation, Dropout

# Definicion del modelo
def modelKeras(activation="relu", optimizer="SGD", drop=1, kernel='random_uniform', neurons=10, moreHiddenLayers=True):
    #definicion de la primera capa con 256 neuronas
    model = Sequential()
    model.add(Dense(256, input_shape=(max_words,), kernel_initializer=kernel,
                bias_initializer=kernel))
    model.add(Dropout(drop))
    model.add(Activation(activation))
    # Se definen mas capas si cumple condicion 
    if moreHiddenLayers == False:
      model.add(Dense(neurons, input_shape=(max_words,), kernel_initializer=kernel,
                  bias_initializer=kernel))
      model.add(Dropout(drop))
      model.add(Activation(activation))
      model.add(Dense(neurons, input_shape=(max_words,), kernel_initializer=kernel,
                  bias_initializer=kernel))
      model.add(Dropout(drop))
      model.add(Activation(activation))
      model.add(Dense(neurons, input_shape=(max_words,), kernel_initializer=kernel,
                  bias_initializer=kernel))
      model.add(Dropout(drop))
      model.add(Activation(activation))
    # Modelo con una sola capa oculta
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
    # Compilacion del modelo se emplea categorical_crossentropy porque tiene varias clases
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    return model

# Una vez definido el modelo se verifica mediante la funcion "summmary()"
modelKeras(moreHiddenLayers=True).summary(70)

______________________________________________________________________
Layer (type)                   Output Shape                Param #    
dense_1 (Dense)                (None, 256)                 256256     
______________________________________________________________________
dropout_1 (Dropout)            (None, 256)                 0          
______________________________________________________________________
activation_1 (Activation)      (None, 256)                 0          
______________________________________________________________________
dense_2 (Dense)                (None, 46)                  11822      
______________________________________________________________________
activation_2 (Activation)      (None, 46)                  0          
Total params: 268,078
Trainable params: 268,078
Non-trainable params: 0
______________________________________________________________________


## Randomized SearchCV

  El wrapper entre __[keras y sklearn](https://keras.io/scikit-learn-api/)__ nos permitio evaluar muchos parametros en nuestra red. 

In [7]:
# Modulos
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [8]:
# Definicion del modelo KerasClassifier
model = KerasClassifier(build_fn=modelKeras, verbose=0)

# Lista de parametros a evaluar
parametros = {
    "activation": ["softmax", "elu", "selu", "softplus", "softsign", "relu" ,"tanh", "sigmoid", "hard_sigmoid", "linear"],
    "optimizer": ["SGD", "RMSprop", "Adagrad", "Adadelta", "Adam", "Adamax", "Nadam"], 
    "drop": [0.2,0.4,0.6,0.8,1],
    "kernel": ["random_normal", "he_uniform", "lecun_normal", "he_normal", "glorot_uniform", "glorot_normal", "lecun_uniform", "zeros", "ones"], 
    "neurons": [5,10,50,100]
    #"moreHiddenLayers": [True, False]
}

# Busqueda aleatoria entre los parametros
random_search = RandomizedSearchCV(estimator=model, param_distributions=parametros)

In [9]:
# Este proceso lleva tiempo por lo que se corre en una linea aparte
# Realiza modelos de manera aleatoria, esto nos permite ver un espectro de cuales paramatros
# y en que rangos se deben fijar los valores.
random_result=random_search.fit(x_train, y_train)

In [ ]:
# La grilla quedo definida con los parametros establecidos anteriormente.
# Si se trabaja en CPU es mejor emplear n_jobs=-1.
random_result

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f0859a8c8d0>,
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'activation': ['softmax', 'elu', 'selu', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'], 'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'], 'drop': [0.2, 0.4, 0.6, 0.8, 1], 'kernel': ['random_normal', 'he_uniform', 'lecun_normal', 'he_normal', 'glorot_uniform', 'glorot_normal', 'lecun_uniform', 'zeros', 'ones']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

Luego de probar las combinaciones usando *RandomizedSearchCV*, tenemos que la configuración de parámetros que nos arroja un mejor desempeño es la siguiente:

In [ ]:
print("Mejor: %f using %s" % (random_result.best_score_, random_result.best_params_))

Mejor: 0.761189 using {'optimizer': 'Adagrad', 'kernel': 'glorot_normal', 'drop': 0.2, 'activation': 'elu'}


In [ ]:
means = random_result.cv_results_['mean_test_score']
stds = random_result.cv_results_['std_test_score']
params = random_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.620129 (0.020941) with: {'optimizer': 'Nadam', 'neurons': 50, 'kernel': 'lecun_uniform', 'drop': 1, 'activation': 'softplus'}
0.592073 (0.029995) with: {'optimizer': 'SGD', 'neurons': 100, 'kernel': 'lecun_normal', 'drop': 0.2, 'activation': 'selu'}
0.001559 (0.000157) with: {'optimizer': 'RMSprop', 'neurons': 50, 'kernel': 'ones', 'drop': 0.2, 'activation': 'relu'}
0.351703 (0.004464) with: {'optimizer': 'Adamax', 'neurons': 5, 'kernel': 'he_uniform', 'drop': 0.8, 'activation': 'sigmoid'}
0.702850 (0.010855) with: {'optimizer': 'Nadam', 'neurons': 10, 'kernel': 'he_uniform', 'drop': 1, 'activation': 'softsign'}
0.618125 (0.004949) with: {'optimizer': 'Adam', 'neurons': 5, 'kernel': 'lecun_uniform', 'drop': 0.2, 'activation': 'tanh'}
0.352260 (0.003917) with: {'optimizer': 'Adadelta', 'neurons': 5, 'kernel': 'lecun_normal', 'drop': 0.8, 'activation': 'linear'}
0.351703 (0.004464) with: {'optimizer': 'Adagrad', 'neurons': 10, 'kernel': 'zeros', 'drop': 0.4, 'activation': 'softmax'}
0.

**Conclusion**: Segun los resultados el mejor set de parametros fueron: 'optimizer': '*Adagrad*', 'kernel': *'glorot_normal*', 'drop': *0.2*, 'activation': '*elu*'. Con un accuracy de **0.76**.

El optimizador **Adagrad**, permite adaptadar los pesos dependiendo de que tan frecuente estos son actualizados. <br>
El inicializador **glorot_normal**, es un inicializador de parametros definido en una Normal truncada entre el numero de pesos de la capa input y la capa output. <br>
El metodo de activacion **elu**, tiene una forma similar a relu, sin embargo esta la zona de cero esta ligeramente por debajo.<br>

Gracias a esto, podemos emplear la funcion *GridSearchCV* para guiarnos con los parametros a evaluar sin sobrecargar el algoritmo.

## Grid SearchCV



Esta busqueda nos permitio realizar todas las combinaciones de los parametros establecidos. Sin embargo, teniendo los resultados anteriores solo se escogieron tres por parametro, ya que requiere mucho procesamiento.

In [ ]:
# Parametros definidos para GridSearchCV
parametros = {
    "activation": ["softmax", "elu", "selu"],
    "optimizer": ["SGD", "RMSprop", "Adagrad"], 
    "drop": [0.2,0.4,0.6],
    "kernel": ["random_normal", "he_uniform", "lecun_normal"]
    #"neurons": [5,10,50,100]
}

# Busqueda de todas las combinaciones posibles entre los parametros
grid = GridSearchCV(estimator=model, param_grid=parametros)

In [ ]:
# De manera similar a RandomSearchCV se ejecuta este comando
# porque resulta en un calculo computacional largo.
grid_result = grid.fit(x_train, y_train)

In [ ]:
# Como en el caso anterior, podemos ver los paramtros evaluados
grid_result

GridSearchCV(cv=None, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f5642e9eef0>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'activation': ['softmax', 'elu', 'selu'], 'optimizer': ['SGD', 'RMSprop', 'Adagrad'], 'drop': [0.2, 0.4, 0.6], 'kernel': ['random_normal', 'he_uniform', 'lecun_normal']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
# Estos resultados se consolidan a continuacion.
# El primer valor corresponde a la media del accuracy y el segundo a su desviacion estandar
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.351703 (0.004464) with: {'activation': 'softmax', 'drop': 0.2, 'kernel': 'random_normal', 'optimizer': 'SGD'}
0.351703 (0.004464) with: {'activation': 'softmax', 'drop': 0.2, 'kernel': 'random_normal', 'optimizer': 'RMSprop'}
0.351703 (0.004464) with: {'activation': 'softmax', 'drop': 0.2, 'kernel': 'random_normal', 'optimizer': 'Adagrad'}
0.351703 (0.004464) with: {'activation': 'softmax', 'drop': 0.2, 'kernel': 'he_uniform', 'optimizer': 'SGD'}
0.351703 (0.004464) with: {'activation': 'softmax', 'drop': 0.2, 'kernel': 'he_uniform', 'optimizer': 'RMSprop'}
0.351703 (0.004464) with: {'activation': 'softmax', 'drop': 0.2, 'kernel': 'he_uniform', 'optimizer': 'Adagrad'}
0.351703 (0.004464) with: {'activation': 'softmax', 'drop': 0.2, 'kernel': 'lecun_normal', 'optimizer': 'SGD'}
0.351703 (0.004464) with: {'activation': 'softmax', 'drop': 0.2, 'kernel': 'lecun_normal', 'optimizer': 'RMSprop'}
0.303941 (0.063309) with: {'activation': 'softmax', 'drop': 0.2, 'kernel': 'lecun_normal', 'opt

Luego de probar las combinaciones usando *GridSearchCV*, tenemos que la configuración de parámetros que tiene un mejor desempeño es la siguiente:

In [ ]:
print("Mejor: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Mejor: 0.704743 using {'activation': 'selu', 'drop': 0.2, 'kernel': 'lecun_normal', 'optimizer': 'Adagrad'}


 **Conclusion**: De acuerdo a los paramatros evaluados tenemos un accuracy de **0.704**. <br> El parametro **selu** (Scaled exponential linear units) es una funcion que mantiene la activacion de la red entre media cero y varianza 1. Algunos estudios han observado como esta funcion ha permitido una convergencia mucho mas rapida con optimizadores como adam.  <br>
Con el caso del optimizador, nuevamente aparece **Adagrad** que los pesos se adapten dependiendo de la frecuencia de actualizacion.  <br>
 
El regularizador **Dropout** de 0.2 corresponde a que exista una probabilidad del 20% para cada neurona de no ser empleada en el calculo. Que sea tan bajo significa que las layers requieren de sufcientes neuronas para procesar la informacion. Adicional, dropout cumple una funcion de regularizador al eliminar "neuronas", que puede verse como eliminar variables.
 
Como era de esperar, los valores de iniciacion (**lecun_normal**) de los pesos (w) deben ser aleatorios para evitar que tengan la misma tendencia en el proceso de aprendizaje.

## Una prueba mas con GridSearchCV
Debido a que no se satisfizo un mejor modelo de acuerdo con los paremetros establecidos. Se procedio a probar unos nuevos.

In [10]:
# Otros parametros definidos para GridSearchCV
parametros = {
    "activation": ["relu" ,"tanh", "sigmoid"],
    "optimizer": ["Adadelta", "Adam", "Adamax"], 
    "drop": [0.2,0.4,0.6],
    "kernel": ["he_normal", "glorot_uniform", "glorot_normal"]
    #"neurons": [5,10,50,100]
}

# Grilla a evaluar el modelo
grid = GridSearchCV(estimator=model, param_grid=parametros)

In [12]:
# Evaluacion de todos los modelos posibles
grid_result = grid.fit(x_train, y_train)

In [13]:
# Grilla obtenida por los paramtros establecidos
grid_result

GridSearchCV(cv=None, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2e2146b7b8>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'activation': ['relu', 'tanh', 'sigmoid'], 'optimizer': ['Adadelta', 'Adam', 'Adamax'], 'drop': [0.2, 0.4, 0.6], 'kernel': ['he_normal', 'glorot_uniform', 'glorot_normal']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
print("Mejor: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Mejor: 0.776776 using {'activation': 'tanh', 'drop': 0.2, 'kernel': 'glorot_normal', 'optimizer': 'Adam'}


**Conclusion**: Se encontraron que los parametros: funcion de activacion **tanh**, y el optimizador **Adam**, tienen un mejor rendimiento para el modelo. De cierta manera las funciones *elu* y *selu* que tienen porciones por debajo de cero, se decir, negativas daban mejores resultados, esto quiere decir que la tanh (con rango entre [-1,1]) esta mas marcado al valor negativo en -1, lo que podria dar un mejor rendimiento al clasificar. Este modelo tuvo un accuracy de **0.78**.